# Mcculloch-Pitts Network

We are going to implement a simple 2 layer network with 7 input neurons and 4 output neurons. This network is designed to recognize the digits 6, 7, 8 and 9. The input neurons are binary neurons, which means that they can only take values 0 or 1. The output neurons are also binary neurons and only the corresponding output neuron for the digit should be 1 and the rest should be 0. For example, if the input is 6, then the output should be [1, 0, 0, 0]. for other combinations (other digits and invalid combinations) the output should be [0, 0, 0, 0].

## Importing Libraries

In [1]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import tqdm

## Creating dataset

 We are going to create a dataset of 128 samples for each combination of input and output. The input is a 7 bit binary number and the output is a 4 bit binary number. 

In [2]:
class SevenSegmentDataset(data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        # self.data should contain [[0,0,0,0,0,0,0],[0,0,0,0,0,0,1],...,[1,1,1,1,1,1,1]]
        self.data = list(itertools.product([0, 1], repeat=7))
        self.data = [list(i) for i in self.data]
        self.labels = [[0, 0, 0, 0] for _ in range(128)]
        # correcting the labels for 6, 7, 8, and 9
        self.labels[95] = [1, 0, 0, 0]
        self.labels[112] = [0, 1, 0, 0]
        self.labels[127] = [0, 0, 1, 0]
        self.labels[123] = [0, 0, 0, 1]
        self.data = torch.tensor(self.data, dtype=torch.float32, requires_grad=True)
        self.labels = torch.tensor(self.labels, dtype=torch.float32, requires_grad=True)
        self.len = len(self.data)
    def __len__(self):
        return self.len
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, label    

In [3]:
dataset = SevenSegmentDataset()
data_loader = data.DataLoader(dataset, batch_size=32, shuffle=True)

## One Layer Network

### Defining the network

In [4]:
class OneLayerNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(7, 4)

    def forward(self, x):
        x = self.linear1(x)
        x = torch.where(x >= 0, torch.tensor([1]), torch.tensor([0]))
        return x

In [5]:
oneLayerNet = OneLayerNet()

### Training the network

In [6]:
oneLayerNet.linear1.weight = nn.Parameter(torch.tensor([[1., -1., 1., 1., 1., 1., 1.],
                                                        [1., 1., 1., -1., -1., -1., -1.],
                                                        [1., 1., 1., 1., 1., 1., 1.],
                                                        [1., 1., 1., 1., -1., 1., 1.]]))

oneLayerNet.linear1.bias = nn.Parameter(torch.tensor([-6., -3., -7., -6.]))


### Evaluating the network

In [7]:
def eval_model(model, data_loader):

    model.eval()

    num_preds = 0
    true_preds = 0

    with torch.no_grad():
        for data, label in data_loader:
            preds = model(data)
            pred_labels = (preds)
            num_preds += len(preds) * 4
            true_preds += (pred_labels == label).sum().item()

            # print wrong predictions
            for i in range(len(preds)):
                if not torch.all(pred_labels[i] == label[i]):
                    print(f'Input: {data[i]}')
                    print(f'Predicted: {pred_labels[i]} | Actual: {label[i]}')
                    print()

    return true_preds, num_preds

In [8]:
oneLayerNet_true_preds, oneLayerNet_num_preds = eval_model(oneLayerNet, data_loader)

print(f'Accuracy: {(oneLayerNet_true_preds/oneLayerNet_num_preds)*100:.2f}')

Accuracy: 100.00


## Two Layer Network

### Defining the network

In [9]:
class TwoLayerNet(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(7, 2)
        self.linear2 = nn.Linear(2, 4)

    def forward(self, x):
        x = self.linear1(x)
        x = torch.where(x >= 0, torch.tensor([1.]), torch.tensor([0.]))
        x = self.linear2(x)
        x = torch.where(x >= 0, torch.tensor([1.]), torch.tensor([0.]))
        return x
    

In [10]:
twoLayerNet = TwoLayerNet()

### Training the network

In [11]:
twoLayerNet.linear1.weight = nn.Parameter(torch.tensor([[1., 1., 0., 0., 0., 1., 1.],
                                                        [0., 0., 1., 1., 1., 0., 1.]]))

twoLayerNet.linear1.bias = nn.Parameter(torch.tensor([-4., -4.]))

In [12]:
twoLayerNet.linear2.weight = nn.Parameter(torch.tensor([[-1., 1.],
                                                        [-1., -1.],
                                                        [1., 1.],
                                                        [1., -1.]]))

twoLayerNet.linear2.bias = nn.Parameter(torch.tensor([-1., 0., -2., -1.]))

### Evaluating the network

In [13]:
digits = [torch.tensor([1, 0, 1, 1, 1, 1, 1], dtype=torch.float32),
            torch.tensor([1, 1, 1, 0, 0, 0, 0], dtype=torch.float32),
            torch.tensor([1, 1, 1, 1, 1, 1, 1], dtype=torch.float32),
            torch.tensor([1, 1, 1, 1, 0, 1, 1], dtype=torch.float32)]

In [14]:
labels = [torch.tensor([1, 0, 0, 0], dtype=torch.float32),
            torch.tensor([0, 1, 0, 0], dtype=torch.float32),
            torch.tensor([0, 0, 1, 0], dtype=torch.float32),
            torch.tensor([0, 0, 0, 1], dtype=torch.float32)]

In [15]:
num_preds = 0
true_preds = 0

for i in range(len(digits)):
    pred = twoLayerNet(digits[i])
    
    num_preds += 4
    true_preds += (pred == labels[i]).sum().item()

    if not torch.all(pred == labels[i]):
        print(f'Digit: {i+6}')
        print(f'Input: {digits[i]}')
        print(f'Predicted: {pred} | Actual: {labels[i]}')
        print()

print(f'Accuracy: {(true_preds/num_preds)*100:.2f}')

Accuracy: 100.00
